<h1 align=center><font size = 5>Analyzing the neighborhoods in the city of Toronto</font></h1>
 

<h1 align=center><font size = 2>Bhuvanesh Selvakumar</font></h1>


## Table of Contents 

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 2>

1.  <a href="#item1">Importing the necessary libraries for analysis</a>

2.  <a href="#item2">Importing open-source data from Wikipedia on Toronto's neighboorhoods and demography</a>

3.  <a href="#item3">Import Toronto's demography dataset from Wikipedia, extract and clean the data</a>

4.  <a href="#item4">Query location data from Foursquare API, and generate  map of Toronto using neighborhood dataset</a>

5.  <a href="#item5">Add Latitude, Longitude and Address of the neighborhoods to the demography dataframe</a>
    </font>
    </div>
    
   

#### 1. Importing the necessary libraries for analysis

In [1]:
# Importing the necessary libraties for the project

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import random
import json
from pandas.io.json import json_normalize
from sklearn import preprocessing
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge geopy --yes
!pip install folium
!pip install geocoder

import folium


#### 2. Importing open-source data from Wikipedia on Toronto's neighboorhoods and demography

In [2]:
# Link to wikipedia neighborhoods
#url_nei = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_nei = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788"
wiki_Toronto_nei = pd.read_html(url_nei)[:]
wiki_Toronto_nei = pd.DataFrame(wiki_Toronto_nei[0])
wiki_Toronto_nei
wiki_Toronto_nei.groupby(by='Borough', axis=0)

#Count the number of boroughs                               
wiki_Toronto_borough = pd.unique(wiki_Toronto_nei['Borough'])
wiki_Toronto_borough_count = wiki_Toronto_nei['Borough'].value_counts()
wiki_Toronto_borough_count

#Reorganize the dataframe and sort them in ascending order
wiki_Toronto_nei = wiki_Toronto_nei.sort_values(by=['Borough'], ascending = True).reset_index()
#wiki_Toronto_nei.reset_index(inplace=True)
wiki_Toronto_nei.drop(['index'], axis=1, inplace=True)
wiki_Toronto_nei.set_index
wiki_Toronto_nei.index.name = 'index'
#wiki_Toronto_nei.index = range(len(wiki_Toronto_nei['Borough']))

#wiki_Toronto_nei
#wiki_Toronto_nei[wiki_Toronto_nei['Postal Code'] == 'M5A']


In [3]:
wiki_Toronto_nei.shape
print('There are {} rows and {} columns in the Toronto neighborhood dataframe'.format(wiki_Toronto_nei.shape[0],
                                                                                       wiki_Toronto_nei.shape[1]))

There are 180 rows and 3 columns in the Toronto neighborhood dataframe


In [4]:
#Getting Latitudes and Longtitudes from Postcodes and integrate it with the neighboorhood dataframe

#!pip install pgeocode
import pgeocode
Toronto_geocoder = pgeocode.Nominatim('ca')

Toronto_boroughs_LL = Toronto_geocoder.query_postal_code(i for i in wiki_Toronto_nei['Postal Code'])[['postal_code',
                                                                                                      'latitude',
                                                                                                      'longitude']]
Toronto_boroughs_LL
wiki_Toronto_nei[['latitude','longitude']] = Toronto_boroughs_LL[['latitude','longitude']]
#wiki_Toronto_nei

#### 3. Import Toronto's demography dataset from Wikipedia, extract and clean the data



In [5]:


def extract_ethnicity(row): 
    y = wiki_Toronto_dem['Ethnicity'][row]
    y = re.search('\(([^)]+)', y).group(1)
    y =  float(y.strip('%'))
    #y =  y.strip('%')

    return y


## Query data of Toronto demograohy from Wikipedia

#!pip install wikipedia #Uncomment this line after installing the first installation of Wikipedia library
import wikipedia as wp
import pandas as pd
import re

url_dem = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
wiki_Toronto_dem = pd.read_html(url_dem)[:]
wiki_Toronto_dem = pd.DataFrame(wiki_Toronto_dem[1])    
#wiki_Toronto_dem.rename(columns={"Second most common language (after English) by name" : "Ethnicity"}, inplace = True)
#wiki_Toronto_dem.groupby(by='Ethnicity', axis=0)

for i in range(2,6):
    x = pd.read_html(url_dem)[:]
    x = pd.DataFrame(x[i])
    wiki_Toronto_dem = wiki_Toronto_dem.append(x, ignore_index=False)

wiki_Toronto_dem.rename(columns={"Second most common language (after English) by name" : "Ethnicity"}, inplace = True)
wiki_Toronto_dem['Ethnicity']
wiki_Toronto_dem.reset_index(inplace=True)

#----------------------------------------------------------------------------------------------------------------------


# Creating a new column and remove unused columns

wiki_Toronto_dem['Ethnicity Percentage (%)'] = ''

for i in range(wiki_Toronto_dem.shape[0]):
    if set(['Map','Census Tracts','Second most common language (after English) by percentage']).issubset(wiki_Toronto_dem):
        wiki_Toronto_dem.drop(columns = ['Map','Census Tracts','Second most common language (after English) by percentage'], axis = 1, inplace=True)
    else:
        pass
    
wiki_Toronto_dem = wiki_Toronto_dem.dropna()

wiki_Toronto_dem.drop(['index'], axis=1, inplace=True)
wiki_Toronto_dem.set_index
wiki_Toronto_dem.index.name = 'index'
wiki_Toronto_dem.index = range(len(wiki_Toronto_dem['Ethnicity']))
wiki_Toronto_dem

#-----------------------------------------------------------------------------------------------------------------------


# Extract the ethnicty and save it in Ethnicity percentage columnn

out = []
for i in range(len(wiki_Toronto_dem['Ethnicity'])):
    out.append(extract_ethnicity(i))

wiki_Toronto_dem['Ethnicity Percentage (%)'] = out
wiki_Toronto_dem['Ethnicity_new'] = wiki_Toronto_dem['Ethnicity'].str.split('(').str[0]

for i in range(len(wiki_Toronto_dem['Ethnicity_new'])):
     wiki_Toronto_dem['Ethnicity_new'][i] = wiki_Toronto_dem['Ethnicity_new'][i].strip()
        
wiki_Toronto_dem
#wiki_Toronto_dem['Ethnicity_new'].value_counts()


#------------------------------------------------------------------------------------------------------------------------

#Create an new column listing the names of neighborhood concatenated with "Toronto" to make the address search easier

temp = wiki_Toronto_dem['Name'] + ', Toronto'
wiki_Toronto_dem.insert(1,'Name-ccat',temp)
wiki_Toronto_dem


#------------------------------------------------------------------------------------------------------------------------

#Clean the neighborhood names (row 1, 55)

pd.set_option('display.max_rows', 500)
wiki_Toronto_dem['Name'][1] = wiki_Toronto_dem['Name'][1].split('/')[0]
wiki_Toronto_dem['Name'][55] = wiki_Toronto_dem['Name'][55].split('/')[0]

wiki_Toronto_dem.head()


C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Name,Name-ccat,FM,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Ethnicity,Ethnicity Percentage (%),Ethnicity_new
0,Crescent Town,"Crescent Town, Toronto",EY,8157,0.40,20393,-10.0,23021,24.5,20.3,Bengali (18.1%),18.1,Bengali
1,Governor's Bridge,"Governor's Bridge/Bennington Heights, Toronto",EY,2112,1.87,1129,4.0,129904,7.1,13.3,Polish (1.4%),1.4,Polish
2,Leaside,"Leaside, Toronto",EY,13876,2.81,4938,3.0,82670,9.7,10.5,Bulgarian (0.4%),0.4,Bulgarian
3,O'Connor–Parkview,"O'Connor–Parkview, Toronto",EY,17740,4.94,3591,-6.1,33517,15.8,19.4,Urdu (3.2%),3.2,Urdu
4,Old East York,"Old East York, Toronto",EY,52220,7.94,6577,-4.6,33172,22.0,19.1,Greek (4.3%),4.3,Greek


#### 4. Query location data from Foursquare API, and generate  map of Toronto using neighborhood dataset



In [6]:

# Use Nominatim function to generate map of Toronto

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="Toronto_explorer")
Toronto_location = geolocator.geocode(address, timeout = None)
Toronto_latitude = Toronto_location.latitude
Toronto_longitude = Toronto_location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(Toronto_latitude, Toronto_longitude))


# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

wiki_Toronto_nei = wiki_Toronto_nei.dropna()


# add markers to map
for lat, lng, label in zip(wiki_Toronto_nei['latitude'], wiki_Toronto_nei['longitude'], wiki_Toronto_nei['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### 5. Add Latitude, Longitude and Address of the neighborhoods to the demography dataframe

In [7]:

# Adding address to the Dataframe 

Toronto_locator =  Nominatim(user_agent="FourSquare_Toronto")
#Toronto_locator.geocode(Toronto_dem_LL[i])
wiki_Toronto_dem['Address'] = ''
wiki_Toronto_dem['Latitude'] = ''
wiki_Toronto_dem['Longitude'] = ''
    
for i in range(len(wiki_Toronto_dem['Name-ccat'])):
    if bool(Toronto_locator.geocode(wiki_Toronto_dem['Name-ccat'][i])) == True:
        wiki_Toronto_dem['Address'][i] = Toronto_locator.geocode(wiki_Toronto_dem['Name-ccat'][i])[0]
    else:
        #return None 
        wiki_Toronto_dem['Address'][i] = 0

#wiki_Toronto_dem



C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
    
#Making a temporary copy of the Dataframe for ease of testing the code

x = wiki_Toronto_dem
x = x[x['Address'] != 0]
x.reset_index(drop=True, inplace=True)
x

def find_LL(row):
    latitude = Toronto_locator.geocode(x['Name-ccat'][row],timeout=None)[1][0]
    longitude = Toronto_locator.geocode(x['Name-ccat'][row],timeout=None)[1][1]
    return latitude, longitude


for i in range(len(x['Name'])):
    x['Latitude'].loc[i] = find_LL(i)[0]
    x['Longitude'].loc[i] = find_LL(i)[1]
    #lati = find_LL(i)[0]
    #long = find_LL(i)[1]
    #x.append({'Latitude':lati,'Longitude':long}, ignore_index=True)

wiki_Toronto_dem = x



C:\Users\Bhuvan\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launc

In [9]:
# Color code the neighborhoods based on the highest secondary ethnicty

neigh_labels = wiki_Toronto_dem['Ethnicity_new'].unique()
color_array = cm.rainbow(np.linspace(0, 1, len(neigh_labels)))
color_array = [colors.rgb2hex(i) for i in color_array]
#color_array

for i in range(len(neigh_labels)):
    wiki_Toronto_dem.loc[wiki_Toronto_dem['Ethnicity_new'] == neigh_labels[i] , "Color_ethnicity"] = color_array[i]
    
#wiki_Toronto_dem


#-----------------------------------------------------------------------------------------------------------------


# Color code the neighborhoods based on the neighborhoods

neigh_labels = wiki_Toronto_dem['FM'].unique()
color_array = cm.rainbow(np.linspace(0, 1, len(neigh_labels)))
color_array = [colors.rgb2hex(i) for i in color_array]
#color_array

for i in range(len(neigh_labels)):
    wiki_Toronto_dem.loc[wiki_Toronto_dem['FM'] == neigh_labels[i] , "Color_FM"] = color_array[i]
    
#wiki_Toronto_dem


In [10]:

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

wiki_Toronto_nei = wiki_Toronto_nei.dropna()


# add markers to map
for lat, lng, label, area, color_ethnicity, color_FM in zip(wiki_Toronto_dem['Latitude'],
                                                      wiki_Toronto_dem['Longitude'],
                                                      wiki_Toronto_dem['Ethnicity_new'],
                                                      wiki_Toronto_dem['Name'],                                                            
                                                      wiki_Toronto_dem['Color_ethnicity'],
                                                      wiki_Toronto_dem['Color_FM']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_FM,
        fill=True,
        fill_color=color_FM,
        fill_opacity=1,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## Analyze the neighborhoods using Foursquare API

#### 1. Input user credentials to access Foursquare API

In [11]:

CLIENT_ID = 'RF0UYLVZDZ3W4IBJOOHUTN3LZZY1YTHCOHODTMLWHSN11HJQ' # your Foursquare ID
CLIENT_SECRET = 'Z1LAGZCWCJOMUQKQ53VXXBCPHGHZD1EOG3O4R0XIFK1RCZYB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RF0UYLVZDZ3W4IBJOOHUTN3LZZY1YTHCOHODTMLWHSN11HJQ
CLIENT_SECRET:Z1LAGZCWCJOMUQKQ53VXXBCPHGHZD1EOG3O4R0XIFK1RCZYB


#### 2. Define a function to query, sort data from Categories list and transfer them into a dataframe



In [12]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#results['response']['groups'][0]['items']


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



# type your answer here
Toronto_venues = getNearbyVenues(names=wiki_Toronto_dem['Name-ccat'],
                                   latitudes=wiki_Toronto_dem['Latitude'],
                                   longitudes=wiki_Toronto_dem['Longitude']
                                  )

#Toronto_venues

Crescent Town, Toronto
Governor's Bridge/Bennington Heights, Toronto
Leaside, Toronto
O'Connor–Parkview, Toronto
Old East York, Toronto
Thorncliffe Park, Toronto
Alderwood, Toronto
Centennial, Toronto
Clairville, Toronto
Eatonville, Toronto
Humber Heights, Toronto
Humberwood, Toronto
Humber Valley Village, Toronto
Islington – Six Points, Toronto
Kingsview Village, Toronto
Long Branch, Toronto
Markland Wood, Toronto
Mimico, Toronto
New Toronto, Toronto
Princess Gardens, Toronto
Agincourt, Toronto
Alexandra Park, Toronto
Allenby, Toronto
Amesbury, Toronto
Armour Heights, Toronto
Banbury, Toronto
Bathurst Manor, Toronto
Bay Street Corridor, Toronto
Bayview Village, Toronto
Bayview Woods – Steeles, Toronto
Bedford Park, Toronto
Bendale, Toronto
Birch Cliff, Toronto
Bloor West Village, Toronto
Bracondale Hill, Toronto
Branson, Toronto
Bridle Path, Toronto
Brockton, Toronto
Cabbagetown, Toronto
Caribou Park, Toronto
Carleton Village, Toronto
Casa Loma, Toronto
Chaplin Estates, Toronto
Christ

In [15]:
#print(Toronto_venues.shape)
Toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt, Toronto",10,10,10,10,10,10
"Alderwood, Toronto",7,7,7,7,7,7
"Alexandra Park, Toronto",100,100,100,100,100,100
"Allenby, Toronto",8,8,8,8,8,8
"Amesbury, Toronto",6,6,6,6,6,6
"Armour Heights, Toronto",3,3,3,3,3,3
"Banbury, Toronto",6,6,6,6,6,6
"Bathurst Manor, Toronto",4,4,4,4,4,4
"Bay Street Corridor, Toronto",100,100,100,100,100,100


#### 2. Apply one-hot encoding

In [16]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot = Toronto_onehot.drop(['Neighborhood'], axis=1)

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 


# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Agincourt, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Alexandra Park, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.010000,0.000000,0.000000,...,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000
3,"Allenby, Toronto",0.000000,0.000000,0.125000,0.125,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Amesbury, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Armour Heights, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Banbury, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bathurst Manor, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Bay Street Corridor, Toronto",0.000000,0.010000,0.000000,0.000,0.0,0.0,0.020000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.020000,0.010000
9,"Bayview Village, Toronto",0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

# Sort the venues in descending order 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


# ==================================================================================================================


num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Toronto_neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']
for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_neighborhoods_venues_sorted.head()

----Agincourt, Toronto----
                   venue  freq
0     Chinese Restaurant   0.2
1             Food Court   0.1
2  Vietnamese Restaurant   0.1
3      Korean Restaurant   0.1
4          Train Station   0.1


----Alderwood, Toronto----
          venue  freq
0   Pizza Place  0.29
1  Skating Rink  0.14
2           Gym  0.14
3           Pub  0.14
4   Coffee Shop  0.14


----Alexandra Park, Toronto----
                    venue  freq
0                     Bar  0.11
1                    Café  0.05
2  Furniture / Home Store  0.05
3    Caribbean Restaurant  0.04
4             Art Gallery  0.02


----Allenby, Toronto----
                venue  freq
0   Fish & Chips Shop  0.12
1   Afghan Restaurant  0.12
2  African Restaurant  0.12
3                Café  0.12
4        Intersection  0.12


----Amesbury, Toronto----
                venue  freq
0                Bank  0.17
1  Athletics & Sports  0.17
2        Intersection  0.17
3                Park  0.17
4         Gas Station  0.17


----Arm

                 venue  freq
0           Playground   0.2
1                 Bank   0.2
2     Business Service   0.2
3                 Park   0.2
4  Arts & Crafts Store   0.2


----Fort York/Liberty Village, Toronto----
           venue  freq
0           Park  0.18
1          Trail  0.12
2          Hotel  0.06
3  Historic Site  0.06
4    Coffee Shop  0.06


----Garden District, Toronto----
            venue  freq
0           Hotel  0.06
1  Clothing Store  0.06
2     Coffee Shop  0.06
3      Restaurant  0.05
4  Cosmetics Shop  0.05


----Glen Park, Toronto----
                 venue  freq
0           Playground   0.2
1               Bakery   0.2
2        Grocery Store   0.2
3  Japanese Restaurant   0.2
4          Pizza Place   0.2


----Governor's Bridge/Bennington Heights, Toronto----
                     venue  freq
0                     Park  0.75
1                    Trail  0.25
2                      ATM  0.00
3                   Museum  0.00
4  North Indian Restaurant  0.00


----G

            venue  freq
0       Pet Store  0.10
1     Coffee Shop  0.10
2  Breakfast Spot  0.05
3     Gas Station  0.05
4    Liquor Store  0.05


----Milliken, Toronto----
                  venue  freq
0    Chinese Restaurant  0.16
1   Japanese Restaurant  0.09
2                Bakery  0.07
3          Intersection  0.05
4  Cantonese Restaurant  0.05


----Mimico, Toronto----
                 venue  freq
0               Bakery  0.25
1                  Bar  0.25
2  American Restaurant  0.25
3         Skating Rink  0.25
4                  ATM  0.00


----Moore Park, Toronto----
               venue  freq
0                Gym  0.50
1  Convenience Store  0.25
2       Tennis Court  0.25
3                ATM  0.00
4             Museum  0.00


----Morningside, Toronto----
                  venue  freq
0           Coffee Shop   0.2
1                  Park   0.2
2  Fast Food Restaurant   0.1
3        Sandwich Place   0.1
4     Mobile Phone Shop   0.1


----Mount Dennis, Toronto----
             

                  venue  freq
0     Indian Restaurant  0.25
1  Caribbean Restaurant  0.17
2           Supermarket  0.08
3   American Restaurant  0.08
4           Coffee Shop  0.08


----Thorncliffe Park, Toronto----
                venue  freq
0   Indian Restaurant  0.11
1   Afghan Restaurant  0.07
2  Turkish Restaurant  0.07
3         Coffee Shop  0.07
4      Sandwich Place  0.07


----Toronto Islands, Toronto----
             venue  freq
0      Music Venue  0.25
1             Café  0.25
2             Park  0.25
3  Harbor / Marina  0.25
4       Nail Salon  0.00


----Trinity–Bellwoods, Toronto----
                   venue  freq
0                 Bakery  0.07
1           Cocktail Bar  0.07
2                   Café  0.05
3  Vietnamese Restaurant  0.05
4              Pet Store  0.05


----Upper Beaches, Toronto----
                  venue  freq
0                  Park  0.33
1  Hungarian Restaurant  0.17
2                  Café  0.17
3           Flower Shop  0.17
4      Asian Restaurant  

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt, Toronto",Chinese Restaurant,Hong Kong Restaurant,Coffee Shop,Food Court,Korean Restaurant,Asian Restaurant,Train Station,Cantonese Restaurant,Vietnamese Restaurant,Farm
1,"Alderwood, Toronto",Pizza Place,Coffee Shop,Sandwich Place,Gym,Skating Rink,Pub,Farmers Market,Farm,Falafel Restaurant,Event Space
2,"Alexandra Park, Toronto",Bar,Café,Furniture / Home Store,Caribbean Restaurant,Gym / Fitness Center,Arepa Restaurant,Boutique,Poutine Place,Bakery,Pizza Place
3,"Allenby, Toronto",Fast Food Restaurant,Café,Afghan Restaurant,African Restaurant,Restaurant,Fish & Chips Shop,Big Box Store,Intersection,Farm,Eastern European Restaurant
4,"Amesbury, Toronto",Intersection,Bank,Athletics & Sports,Coffee Shop,Gas Station,Park,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


### Identify neighborhoods with active Tamil population and evaluate if they have any Indian restaurants

In [104]:
#Narrow down on neighborhoods with active Indian restaurants

Toronto_neighborhoods_venues_sorted
len(Toronto_grouped[Toronto_grouped["Indian Restaurant"] > 0])
Indian_restaurant_nei = Toronto_grouped[["Neighborhood","Indian Restaurant"]]
x=wiki_Toronto_dem.sort_values(['Name'], ascending=True).reset_index()
x.drop(['index'], axis=1, inplace=True)
x

Indian_restaurant_nei
len(Indian_restaurant_nei)
len(x)

y = x[x['Name-ccat'].isin(Indian_restaurant_nei['Neighborhood'])].reset_index()
y.drop(['index'], axis=1, inplace=True)
y

z = pd.DataFrame(Indian_restaurant_nei['Neighborhood'])

z['Indian restaurant freq'] = Indian_restaurant_nei['Indian Restaurant']
z['Ethnicity'] = y['Ethnicity']
z['Ethnicity Percentage (%)'] = y['Ethnicity Percentage (%)']
z['Ethnicity_new'] = y['Ethnicity_new']
z['Latitude'] = y['Latitude']
z['Longitude'] = y['Longitude']
z['Color_ethnicity'] = y['Color_ethnicity']
z['Color_FM'] = y['Color_FM']
z

#z['Indian restaurant freq']
z1 = z[z['Ethnicity_new'] == 'Tamil'].sort_values(by = ['Ethnicity Percentage (%)'], ascending=True)
z1.reset_index(inplace=True)
z1.drop(['index'], axis=1, inplace=True)
z1

#Identify neighborhoods which dont have any Indian restaurants
z2 = z1[z1['Indian restaurant freq'] == 0].sort_values(by = ['Ethnicity Percentage (%)'], ascending=False)
z2.reset_index(inplace=True)
z2.drop(['index'], axis=1, inplace=True)
z2.head()
#z2.sort_values(by = ['Ethnicity Percentage (%)'], ascending=False)

,Neighborhood,Indian restaurant freq,Ethnicity,Ethnicity Percentage (%),Ethnicity_new,Latitude,Longitude,Color_ethnicity,Color_FM
0,"Rouge Hill, Toronto",0.0,Tamil (15.6%),15.6,Tamil,43.8049,-79.1658,#a6f89d,#4df3ce
1,"Malvern, Toronto",0.0,Tamil (12.2%),12.2,Tamil,43.8092,-79.2217,#a6f89d,#4df3ce
2,"Scarborough Village, Toronto",0.0,Tamil (11.4%),11.4,Tamil,43.7437,-79.2116,#a6f89d,#4df3ce
3,"Morningside, Toronto",0.0,Tamil (10.8%),10.8,Tamil,43.7826,-79.205,#a6f89d,#4df3ce
4,"Scarborough City Centre, Toronto",0.0,Tamil (10.3%),10.3,Tamil,43.717,-79.2547,#a6f89d,#4df3ce


In [103]:
#Generate range of colors for based on % of Tamils in each of the above neighborhood

'''import colorsys
N = 5
HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)

HSV_tuples = [(0, 1-x/100, 0) for x in z2['Ethnicity Percentage (%)']]
RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)

print(HSV_tuples)

from sklearn import preprocessing
x_array = np.array(z2['Ethnicity Percentage (%)'])
x_array = x_array/100
x_array

normalized_arr = preprocessing.normalize([x_array])
print(normalized_arr)

x_array = ((x_array-min(x_array))/(max(x_array) - min(x_array)))
x_array
RGB_x_array = map(lambda x: colorsys.hsv_to_rgb(*x), x_array)
RGB_x_array

r1, g1, b1 = [1,0,0]
r2, g2, b2 = [0,1,0]
rdelta, gdelta, bdelta = (r2-r1)/steps, (g2-g1)/steps, (b2-b1)/steps
for step in len(z2['Ethnicity Percentage (%)']):
    r1 += rdelta
    g1 += gdelta
    b1 += bdelta
    output.append((r1, g1, b1))'''
    

"import colorsys\nN = 5\nHSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]\nRGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)\n\nHSV_tuples = [(0, 1-x/100, 0) for x in z2['Ethnicity Percentage (%)']]\nRGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)\n\nprint(HSV_tuples)\n\nfrom sklearn import preprocessing\nx_array = np.array(z2['Ethnicity Percentage (%)'])\nx_array = x_array/100\nx_array\n\nnormalized_arr = preprocessing.normalize([x_array])\nprint(normalized_arr)\n\nx_array = ((x_array-min(x_array))/(max(x_array) - min(x_array)))\nx_array\nRGB_x_array = map(lambda x: colorsys.hsv_to_rgb(*x), x_array)\nRGB_x_array\n\nr1, g1, b1 = [1,0,0]\nr2, g2, b2 = [0,1,0]\nrdelta, gdelta, bdelta = (r2-r1)/steps, (g2-g1)/steps, (b2-b1)/steps\nfor step in len(z2['Ethnicity Percentage (%)']):\n    r1 += rdelta\n    g1 += gdelta\n    b1 += bdelta\n    output.append((r1, g1, b1))"

### Implement k-means clusterning to develop neighborhood clusters with Indian restaurants

In [130]:
#wiki_Toronto_dem
Toronto_grouped

# set number of clusters
kclusters = 3

#Indian_restaurant_nei_clustering = Indian_restaurant_nei.drop('Neighborhood', 1)
Indian_restaurant_nei_clustering = pd.DataFrame(z['Indian restaurant freq'])
Indian_restaurant_nei_clustering
Indian_restaurant_nei

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1)
kmeans.fit_transform(Indian_restaurant_nei_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

z['Cluster label'] = kmeans.labels_
Indian_restaurant_nei['Cluster label'] = kmeans.labels_
Indian_restaurant_nei

Indian_restaurant_nei_final = Indian_restaurant_nei.join(Toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(Indian_restaurant_nei_final.shape)
Indian_restaurant_nei_final.head()
#Indian_restaurant_nei_final['Cluster label']
#z[z['Cluster label'] == 2]

z.head()



(3534, 9)


C:\Users\Bhuvan\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Neighborhood,Indian restaurant freq,Ethnicity,Ethnicity Percentage (%),Ethnicity_new,Latitude,Longitude,Color_ethnicity,Color_FM,Cluster label
0,"Agincourt, Toronto",0.0,Cantonese (19.3%),19.3,Cantonese,43.7854,-79.2785,#58f8c9,#4df3ce,0
1,"Alderwood, Toronto",0.0,Polish (6.2%),6.2,Polish,43.6017,-79.5452,#6e1cff,#1996f3,0
2,"Alexandra Park, Toronto",0.0,Cantonese (17.9%),17.9,Cantonese,43.6508,-79.4043,#58f8c9,#b2f396,0
3,"Allenby, Toronto",0.0,Russian (1.4%),1.4,Russian,43.7114,-79.5534,#6dfdbf,#b2f396,0
4,"Amesbury, Toronto",0.0,Spanish (6.1%),6.1,Spanish,43.7062,-79.4835,#08bee9,#ff964f,0


In [128]:
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude],zoom_start=14)

# set color scheme for the clusters

'''# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster, label in zip(Indian_restaurant_nei_final['Neighborhood Latitude'],
                             Indian_restaurant_nei_final['Neighborhood Longitude'],
                             Indian_restaurant_nei_final['Cluster label'],
                             Indian_restaurant_nei_final['Cluster label']):
    
    label = folium.Popup('Cluster no: {}'.format(str(label)), parse_html=True)
    folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup = label,
            color =markers_colors[cluster],
            fill = True,
            fill_color=markers_colors[cluster],
            fill_opacity=1
            ).add_to(map_clusters)
    
map_clusters'''



# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster, label in zip(z['Latitude'],
                             z['Longitude'],
                             z['Cluster label'],
                             z['Cluster label']):
    
    label = folium.Popup('Cluster no: {}'.format(str(label)), parse_html=True)
    folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup = label,
            color =markers_colors[cluster],
            fill = True,
            fill_color=markers_colors[cluster],
            fill_opacity=1
            ).add_to(map_clusters)
    
map_clusters

In [144]:

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

wiki_Toronto_nei = wiki_Toronto_nei.dropna()


# add markers to map
for lat, lng, label, area, color_ethnicity, color_FM in zip(wiki_Toronto_dem['Latitude'],
                                                      wiki_Toronto_dem['Longitude'],
                                                      wiki_Toronto_dem['Ethnicity_new'],
                                                      wiki_Toronto_dem['Name'],                                                            
                                                      wiki_Toronto_dem['Color_ethnicity'],
                                                      wiki_Toronto_dem['Color_FM']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_FM,
        fill=True,
        fill_color=color_FM,
        fill_opacity=1,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto



z1
z2 = z1[z1['Indian restaurant freq'] == 0]
z2


# Overlay Tamil restaurants atop the neighborhoods

for lat, lng, label, neighborhood in zip(z2['Latitude'],
                                         z2['Longitude'],
                                         z2['Ethnicity'],
                                         z2['Neighborhood']):
    label = folium.Popup('Location: {}, Ethnicity: {}'.format(neighborhood, label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0).add_to(map_Toronto) 
     
    
map_Toronto


In [ ]:
z1
z2 = z1[z1['Indian restaurant freq'] == 0].reset_index()
z2.drop(['index'], axis=1, inplace=True)
z2

In [ ]:
#wiki_Toronto_dem
Toronto_grouped

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Uncomment the following line of code if running this code for the second time
#Toronto_neighborhoods_venues_sorted = Toronto_neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1) 
Toronto_neighborhoods_venues_sorted


# add clustering labels
Toronto_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_neighborhoods_venues_sorted.head()
Toronto_merged = wiki_Toronto_dem

Toronto_merged = Toronto_merged.rename(columns = {'Name':'Neighborhood'})
Toronto_merged = Toronto_merged.sort_values(['Neighborhood'], ascending=True).reset_index(drop=True)
Toronto_merged

# merge manhattan_grouped with Toronto_merged data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name-ccat')

Toronto_merged.head() # check the last columns!
Toronto_merged.rename(columns = {'FM':'Borough'})

Toronto_merged = Toronto_merged.dropna().reset_index(drop=True)

In [ ]:
# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'],
                                  Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters


In [ ]:
Toronto_merged.shape

### Analyze Cluster #1

In [ ]:
#Analyse Cluser #1:

test = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(19, Toronto_merged.shape[1]))]]
test

#Find the top 3 places in each ranking list:

"""for column in test.columns[1:]:
    print('------------{}------------'.format(column))
    print(test[column].value_counts().head(3))
    print('\n')"""

#Extract columns 
x = test.iloc[:,1:]
x.mode().iloc[0,:]    

#Wuery the list of all activities
activities = Toronto_grouped.columns
activities

Cluster1_all = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False)
Cluster1_top10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).head(10)
Cluster1_bottom10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).tail(10)
Cluster1_top10


print('----------- CLUSTER#1 TOP 10 -----------')
#print('\n')
print(Cluster1_top10)
print('\n')


print('----------- CLUSTER#1 BOTTOM 10 -----------')
#print('\n')
print(Cluster1_bottom10)



# Count the frequency of occurence of every event
Cluster1_all
y = pd.DataFrame(Cluster1_all,columns =['Freq'])
y.index.name = 'Event'
y

# Filter the popular cuisine (restaurants)
filter_restaurants = y[y.index.str.contains('Rest')]
filter_restaurants

# Filter the popular shops 
filter_shops = y[y.index.str.contains('Shop','Store')]
#filter_shops

filter_restaurants
Cluster1_all

#### Analyzing cluster #1 we learn that:
1. Poeple in cluster #1 enjoy visiting park and event spaces the most.
2. Ethiopian and Falafel restaurants tend to be very popular among the populace, and there are limited dining options for Asian and Caribbean cuisine
3. Limited interest in fine arts: Galleries, music venues, piano bar
4. Electronics shopping tends to be popular among the popuation, though the overall interest in shopping is very limited. Therefore, we can safely assume that the neighborhoods are dominantly commercial-residential blocks
5. Good neighborhood for sports/outdoor activities: Skating rink, tennis, golf, baseball, playground

### Analyze Cluster #2

In [ ]:


test = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(19, Toronto_merged.shape[1]))]]

#Find the top 3 places in each ranking list
"""for column in test.columns[1:]:
    print('------------{}------------'.format(column))
    print(test[column].value_counts().head(3))
    print('\n')"""
    
    

#Extract columns 
x = test.iloc[:,1:]
x.mode().iloc[0,:]    

#Wuery the list of all activities
activities = Toronto_grouped.columns
activities

Cluster1_all = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False)
Cluster1_top10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).head(10)
Cluster1_bottom10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).tail(10)
Cluster1_top10


print('----------- CLUSTER#1 TOP 10 -----------')
#print('\n')
print(Cluster1_top10)
print('\n')


print('----------- CLUSTER#1 BOTTOM 10 -----------')
#print('\n')
print(Cluster1_bottom10)
print('\n')


# Count the frequency of occurence of every event
Cluster1_all
y = pd.DataFrame(Cluster1_all,columns =['Freq'])
y.index.name = 'Event'
y

# Filter the popular cuisine (restaurants)
filter_restaurants = y[y.index.str.contains('Rest')]
filter_restaurants

# Filter the popular shops 
filter_shops = y[y.index.str.contains('Shop','Store')]
#filter_shops

filter_restaurants
Cluster1_all


#### Analyzing cluster #2 we learn that:
1. Yoga studios, parks, Farmers market, event spaces tend to be the popular go-to spot
2. Access to trails and rivers
3. Fast food, Falafel and Ethiopian restaurants are popular; Eastern European restuarants are relatively less popular

### Analyze Cluster #3


In [ ]:
#Analyse Cluser #3

test = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(19, Toronto_merged.shape[1]))]]


#Find the top 3 places in each ranking list
"""for column in test.columns[1:]:
    print('------------{}------------'.format(column))
    print(test[column].value_counts().head(3))
    print('\n')"""
    
    

#Extract columns 
x = test.iloc[:,1:]
x.mode().iloc[0,:]    

#Wuery the list of all activities
activities = Toronto_grouped.columns
activities

Cluster1_all = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False)
Cluster1_top10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).head(10)
Cluster1_bottom10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).tail(10)
Cluster1_top10


print('----------- CLUSTER#1 TOP 10 -----------')
#print('\n')
print(Cluster1_top10)
#print('\n')


print('----------- CLUSTER#1 BOTTOM 10 -----------')
#print('\n')
print(Cluster1_bottom10)

# Count the frequency of occurence of every event
Cluster1_all
y = pd.DataFrame(Cluster1_all,columns =['Freq'])
y.index.name = 'Event'
y

# Filter the popular cuisine (restaurants)
filter_restaurants = y[y.index.str.contains('Rest')]
filter_restaurants

# Filter the popular shops 
filter_shops = y[y.index.str.contains('Shop','Store')]
filter_shops
Cluster1_all

#### Analyzing cluster #3 we learn that:
1. Coffee shops and cafes are the most popular eateries
2. Lot of options for dining and arts

### Analyze Cluster #4


In [ ]:
#Analyse Cluser #4

test = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(19, Toronto_merged.shape[1]))]]


#Find the top 3 places in each ranking list
"""for column in test.columns[1:]:
    print('------------{}------------'.format(column))
    print(test[column].value_counts().head(3))
    print('\n')"""
    
    

#Extract columns 
x = test.iloc[:,1:]
x.mode().iloc[0,:]    

#Wuery the list of all activities
activities = Toronto_grouped.columns
activities

Cluster1_all = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False)
Cluster1_top10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).head(10)
Cluster1_bottom10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).tail(10)
Cluster1_top10


print('----------- CLUSTER#1 TOP 10 -----------')
#print('\n')
print(Cluster1_top10)
#print('\n')


print('----------- CLUSTER#1 BOTTOM 10 -----------')
#print('\n')
print(Cluster1_bottom10)



# Count the frequency of occurence of every event
Cluster1_all
y = pd.DataFrame(Cluster1_all,columns =['Freq'])
y.index.name = 'Event'
y

# Filter the popular cuisine (restaurants)
filter_restaurants = y[y.index.str.contains('Rest')]
filter_restaurants

# Filter the popular shops 
filter_shops = y[y.index.str.contains('Shop','Store')]
filter_shops

filter_restaurants
#Cluster1_all


#### Analyzing cluster #4 we learn that:
1. Coffee shops tend to be the popular go-to spot
2. Pizza, fast-food and Ethiopian restaurants tend to be very popular among the populace, with limited interest in French, Mediterranean, Middle Eastern, Portugese and Ramen restaurants
4. Likely a residential cluster as there are manny grocery stores and pharmacies in this cluster (popular signatures of residential neighborhoods)

### Analyze Cluster #5


In [ ]:
#Analyse Cluser #5

test = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(19, Toronto_merged.shape[1]))]]


#Find the top 3 places in each ranking list
"""for column in test.columns[1:]:
    print('------------{}------------'.format(column))
    print(test[column].value_counts().head(3))
    print('\n')"""
    
    

#Extract columns 
x = test.iloc[:,1:]
x.mode().iloc[0,:]    

#Wuery the list of all activities
activities = Toronto_grouped.columns
activities

Cluster1_all = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False)
Cluster1_top10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).head(10)
Cluster1_bottom10 = x.apply(pd.Series.value_counts).sum(axis=1).sort_values(ascending=False).tail(10)
Cluster1_top10


print('----------- CLUSTER#1 TOP 10 -----------')
#print('\n')
print(Cluster1_top10)
#print('\n')


print('----------- CLUSTER#1 BOTTOM 10 -----------')
#print('\n')
print(Cluster1_bottom10)


# Count the frequency of occurence of every event
Cluster1_all
y = pd.DataFrame(Cluster1_all,columns =['Freq'])
y.index.name = 'Event'
y

# Filter the popular cuisine (restaurants)
filter_restaurants = y[y.index.str.contains('Rest')]
filter_restaurants

# Filter the popular shops 
filter_shops = y[y.index.str.contains('Shop','Store')]
filter_shops

filter_restaurants
Cluster1_all

#### Analyzing cluster #5 we learn that:
1. Active for outdoor/sporting activities: Field, trail, Hockey arena, Basket ball court

In [ ]:
wiki_Toronto_dem[wiki_Toronto_dem['Name'] == ]

In [ ]:
#Toronto_grouped
#Toronto_venues['Venue Category'].value_counts()[Toronto_venues['Venue Category'].value_counts().index.str.contains('Rest')]

#Toronto_grouped
#wiki_Toronto_dem
#Toronto_venues


In [ ]:

test = Toronto_venues[Toronto_venues['Venue Category'] == 'Indian Restaurant']
test

# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi in zip(test['Neighborhood Latitude'], test['Neighborhood Longitude'],test['Neighborhood']):
    label = folium.Popup(poi, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= 'red', #rainbow[int(cluster)],
        fill=True,
        fill_color= 'red', 
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [ ]:
test = Toronto_venues[Toronto_venues['Venue Category'] == 'Indian Restaurant']
test.reset_index(drop =True)